In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
data = pd.read_csv('Changan.csv')

### 1. 数据清洗
- 挑选需要的字段：
- 数据类型转换

#### 1.1 挑选字段

In [6]:
data.columns

Index(['_id', 'content', 'createTime', 'downCount', 'id', 'parentCommentId',
       'picList', 'replyCount', 'upCount', 'user.avatarLarge',
       'user.avatarMiddle', 'user.avatarSmall', 'user.userCode', 'user.userId',
       'user.userLevel', 'user.userName', 'user.vipInfo.icon',
       'user.vipInfo.mmid', 'user.vipInfo.name', 'user.vipInfo.state',
       'user.vipInfo.vipGrade', 'userId', 'userIsLiked', 'episode'],
      dtype='object')

In [7]:
col_keep = ['id', 'userId', 'content', 'upCount', 'downCount', 'replyCount', 'createTime',
           'user.userName', 'user.userLevel', 'user.userCode', 'user.vipInfo.mmid',
           'user.vipInfo.name', 'user.vipInfo.vipGrade', 'episode']
data = data[col_keep]

#### 1.2 数据类型转换

In [8]:
data.sample(5)

,id,userId,content,upCount,downCount,replyCount,createTime,user.userName,user.userLevel,user.userCode,user.vipInfo.mmid,user.vipInfo.name,user.vipInfo.vipGrade,episode
9131,1000008736735,1363337817,我都不知道唉,0,0,1,1561645476802,大子233,95,UNTQ1MzM1MTI2OA==,NaN,NaN,NaN,1
3477,1000009790661,1202283170,抖音来的,0,0,0,1561813658990,杨杨晓晓青青,16,UNDgwOTEzMjY4MA==,NaN,NaN,NaN,1
929,1000011523394,1248480610,为千玺而來，也被雷佳音圈粉,3,0,1,1562259859777,罗瑟琳君8_8,20,UNDk5MzkyMjQ0MA==,100002.0,优酷土豆黄金会员,2.0,16
56,1000011679610,1238857232,我暂停了电视，我只想说一句为什么好多人说此剧不好看呢？我想问问你们走心了吗？用心看了吗？我也...,31,0,1,1562310881162,OwO迪基i,100,UNDk1NTQyODkyOA==,100002.0,优酷土豆黄金会员,4.0,4
694,1000010387818,1176689253,好[真棒],0,0,0,1561949320207,高浮尘,28,UNDcwNjc1NzAxMg==,100002.0,优酷土豆黄金会员,5.0,2


In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32903 entries, 0 to 1355
Data columns (total 14 columns):
id                       32903 non-null int64
userId                   32903 non-null int64
content                  32903 non-null object
upCount                  32903 non-null int64
downCount                32903 non-null int64
replyCount               32903 non-null int64
createTime               32903 non-null int64
user.userName            32903 non-null object
user.userLevel           32903 non-null int64
user.userCode            32903 non-null object
user.vipInfo.mmid        21701 non-null float64
user.vipInfo.name        21701 non-null object
user.vipInfo.vipGrade    21701 non-null float64
episode                  32903 non-null object
dtypes: float64(2), int64(7), object(5)
memory usage: 3.8+ MB


In [10]:
# createTime字段
data['bj_time'] = pd.to_datetime(data['createTime'].values, unit='ms', utc=True).tz_convert('Asia/Shanghai')

In [11]:
data['emoji'] = data['content'].str.extract("\[(.*?)\]")

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  """Entry point for launching an IPython kernel.


### 2. 问题
- 该剧总体评价如何？
- 该剧有何亮眼之处？
- 为什么有些观众不喜欢这部剧（台词、叙事、细节）？
- 大家对于雷佳音和易烊千玺在戏中的评价如何？
- 口碑是从什么时候变差的？

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32903 entries, 0 to 32902
Data columns (total 18 columns):
id                       32903 non-null int64
userId                   32903 non-null int64
content                  32903 non-null object
upCount                  32903 non-null int64
downCount                32903 non-null int64
replyCount               32903 non-null int64
createTime               32903 non-null int64
user.userName            32903 non-null object
user.userLevel           32903 non-null int64
user.userCode            32903 non-null object
user.vipInfo.mmid        21701 non-null float64
user.vipInfo.name        21701 non-null object
user.vipInfo.vipGrade    21701 non-null float64
episode                  32903 non-null int64
bj_time                  32903 non-null object
emoji                    4282 non-null object
sentiment                32903 non-null float64
bd_sent                  32840 non-null float64
dtypes: float64(4), int64(8), object(6)
memory usa

In [5]:
data.sample(5)

,id,userId,content,upCount,downCount,replyCount,createTime,user.userName,user.userLevel,user.userCode,user.vipInfo.mmid,user.vipInfo.name,user.vipInfo.vipGrade,episode,bj_time,emoji,sentiment,bd_sent
32737,1000011857457,1125381111,明年能看完？,0,0,1,1562251504592,无名男神123,39,UNDUwMTUyNDQ0NA==,100002.0,优酷土豆黄金会员,6.0,16,2019-07-04 22:45:04.592000+08:00,NaN,0.953433,0.465506
16663,1000008717126,1217419314,期待期待,0,0,0,1561638030951,只为鹤,8,UNDg2OTY3NzI1Ng==,NaN,NaN,NaN,1,2019-06-27 20:20:30.951000+08:00,NaN,0.815406,0.732587
12826,1000008726542,1348457965,终于等到你，我的演员大佬！,0,0,1,1561639831794,陈倩58805,7,UNTM5MzgzMTg2MA==,100002.0,优酷土豆黄金会员,2.0,1,2019-06-27 20:50:31.794000+08:00,NaN,0.985300,0.581345
26575,1000009679773,455442546,第八集，有闻染读成染闻,0,0,1,1561798202156,吚吷,66,UMTgyMTc3MDE4NA==,100002.0,优酷土豆黄金会员,6.0,8,2019-06-29 16:50:02.156000+08:00,NaN,0.377323,0.555884
1854,1000010774716,373716399,剧情不怎么样,3,1,0,1562040939381,后来时间都与我,15,UMTQ5NDg2NTU5Ng==,NaN,NaN,NaN,1,2019-07-02 12:15:39.381000+08:00,NaN,0.474806,0.147864


#### 2.1 该剧总体评价如何？

In [28]:
from aip import AipNlp


APP_ID = ''
API_KEY = ''
SECRET_KEY = ''

client = AipNlp(APP_ID, API_KEY, SECRET_KEY)


def get_sent_baidu(data):
    return_data = client.sentimentClassify(data)
    items = return_data.get('items')  
    if items:
        return items[0]['positive_prob']

In [39]:
data['bd_sent'] = data['content'].apply(get_sent_baidu)
data.to_csv('/Users/apple/Desktop/Changan.csv', index=False)

In [11]:
sns.distplot(data.loc[data['bd_sent']>0, 'bd_sent'], kde=False)

<Figure size 432x288 with 1 Axes>

In [10]:
data['bd_sent'].mean()

0.5709787899768576

In [9]:
data.loc[(data['bd_sent']<0.4)|(data['bd_sent']>0.6), 'bd_sent'].mean()

0.5965854987422653

In [13]:
from pyecharts import Bar

ERROR:lml.utils:failed to import pyecharts_snapshot
Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/site-packages/lml/utils.py", line 43, in do_import
    plugin_module = __import__(plugin_module_name)
ModuleNotFoundError: No module named 'pyecharts_snapshot'


In [16]:
bar = Bar("《长安十二时辰》各大平台评分", width = 500,height=500)
bar.add("", ['优酷','豆瓣','IMDB','时光网'], [8.8, 8.6, 8.5, 8.4], is_stack=True,bar_category_gap='40%', 
       xaxis_label_textsize=20, yaxis_label_textsize=14, is_label_show=True)
bar

In [65]:
emoji = data['emoji'].value_counts()[:15]

In [66]:
from pyecharts import Polar


polar = Polar(" ", width=1200, height=600)
polar.add(
    "",
    emoji.values,
    angle_data=emoji.index,
    type="barAngle",
    is_stack=True,
)

polar

#### 2.2 该剧有何亮眼之处？

In [62]:
data.loc[data['bd_sent'] > 0.7, ['content', 'bd_sent', 'upCount']].sort_values(by='upCount', ascending=False)[:10]

,content,bd_sent,upCount
6629,真的太感谢官方了，还没有官宣就放了，给了千纸鹤一个大大的福利啊，6.27的小北没有，但是给了...,0.802730,7353
2928,关于唐代的治安：\n 靖安司是虚构的机构，唐代长安的治安机构按照保卫地点的不同有不同的...,0.957280,4953
12892,虽然没有宣传但是也没关系，酒香不怕巷子深，好的东西肯定能被人看见的。,0.714026,3892
4415,希望啊鹤儿们要理智，在看剧的时候不要老是刷千玺，那样会给千玺招黑的，咱们乖乖看剧，认真评价，...,0.790711,3322
2991,各位“长安小主”：\n 为给大家详解剧中遇到的历史知识点，我们特意请来陕西师范大学历史...,0.723197,2594
20645,剧情非常烧脑，紧凑不拖拉，电影般的质感，视觉效果很舒服，张小敬和少年李必形象鲜明深刻，期待两...,0.914036,2205
14467,期待已久的剧终于播出了，谢谢剧组工作人员的全体付出，原音好评！制作精良，疯狂的为主创演员们打...,0.931328,1744
6558,好剧不怕晚，谢谢优酷没让我们白等！我千的剧支持起来！祝长安十二时辰大火！,0.972783,1727
20058,摄影师的长镜头真的是超级棒了，而且滤镜也很舒服，很少有这么舒服的滤镜了，感觉是个大制作，演员...,0.993490,1615
12569,什么时候开播的呀，一直考试，今天才看到，猝不及防的落泪，真的期待很久了[惊哭],0.786992,1391


In [72]:
import jieba
from collections import Counter
from pyecharts import WordCloud

jieba.add_word('服道化')
jieba.add_word('良心剧')
jieba.add_word('易烊千玺')
jieba.add_word('雷佳音')
jieba.add_word('千玺')
jieba.add_word('质感')
jieba.add_word('长安十二时辰')
jieba.add_word('四字弟弟')

swords = [x.strip() for x in open ('/home/kesci/input/Changan6837/stopwords.txt')]

In [73]:
def plot_word_cloud(data, swords):
    text = ''.join(data)
    words = list(jieba.cut(text))
    ex_sw_words = []
    for word in words:
        if len(word)>1 and (word not in swords):
            ex_sw_words.append(word)
    c = Counter()
    c = Counter(ex_sw_words)
    wc_data = pd.DataFrame({'word':list(c.keys()), 'counts':list(c.values())}).sort_values(by='counts', ascending=False).head(100)
    wordcloud = WordCloud(width=1300, height=620)
    wordcloud.add("", wc_data['word'], wc_data['counts'], word_size_range=[20, 100])
    return wordcloud

In [74]:
plot_word_cloud(data.loc[data['bd_sent']>0.7, 'content'], swords)

#### 2.3 为什么有些观众不喜欢这部剧（台词、叙事、细节）？

In [81]:
data.loc[data['bd_sent']<0.3, ['bd_sent', 'content']].sample(5)

,bd_sent,content
4474,0.195073,昆仑奴是陈建斌给配的么？？皇上你不是应该在陪嬛嬛么
6290,0.076694,我看不懂这电视剧。[勉强]没文化好可怕！
25834,0.295053,秦器感觉特别欠打
31626,0.080976,这个剧非常好看，就是编剧有很多地方不合理，绑架的时候手都够得着嘴了，无语，还有通传杀徐斌那段...
11741,0.090634,太突然了，有点没缓过来？


In [82]:
jieba.add_word('烧脑')

In [83]:
plot_word_cloud(data.loc[data['bd_sent']<0.3, 'content'], swords)

In [87]:
data.loc[(data['bd_sent']<0.3)&(data['content'].str.contains('勉强')), ['bd_sent', 'content']].sample(5)

,bd_sent,content
29301,0.103478,衣服太美了[花心][花心]话说 有组团偷衣服的没[勉强][勉强]
17738,0.076010,什么情况啊[勉强][勉强]
20453,0.034718,我真的很想看懂，可是智商跟不上啊，撤了姐们儿[勉强][勉强][勉强]
5425,0.101581,看来又要为大佬充会员了[勉强][勉强]
21362,0.246930,[勉强]文言文看不明白


In [108]:
data.loc[data['bd_sent']<0.3, ['bd_sent', 'content']].sample(5)

,bd_sent,content
32339,0.068424,原来不是显示25集嘛，怎么煮水到48了
14106,0.278151,什么时候出的亚连热搜都没有，千玺也没宣传
30581,0.245948,我怎么发现总集篇变多了，我又幻觉了吗
27045,0.028253,我堂堂的不良帅张小敬，在极挑里竟然被几个鳖孙整得爹妈都不认识[勉强][勉强][勉强][勉强]
21148,0.246471,后悔当初没好好学文言文了


In [152]:
data['bj_time'] = pd.to_datetime(data['bj_time'])

In [163]:
data['bj_time'].dt.date.value_counts()

2019-06-27    12980
2019-06-28     4358
2019-07-04     2969
2019-06-29     2587
2019-07-05     2170
2019-06-30     1724
2019-07-06     1713
2019-07-01     1505
2019-07-02     1383
2019-07-03     1322
2019-07-07      192
Name: bj_time, dtype: int64

#### 2.4大家对于雷佳音和易烊千玺在戏中的评价如何？

In [184]:
data['ljy'] = data['content'].str.contains('雷佳音|大头|张小敬')
data['yyqx'] = data['content'].str.contains('千玺|四字|李必')

In [187]:
data.loc[data['ljy'], 'bd_sent'].mean()

0.6100079751508621

In [188]:
data.loc[data['yyqx'], 'bd_sent'].mean()

0.6359185166758544

In [189]:
plot_word_cloud(data.loc[data['ljy'], 'content'], swords)

In [194]:
plot_word_cloud(data.loc[(data['ljy'])&(data['bd_sent']>0.7), 'content'], swords)

In [190]:
plot_word_cloud(data.loc[data['yyqx'], 'content'], swords)

In [191]:
plot_word_cloud(data.loc[(data['yyqx'])&(data['bd_sent']<0.3), 'content'], swords)

In [192]:
data.loc[(data['yyqx'])&(data['bd_sent']<0.3), ['content', 'bd_sent']].sample(5)

,content,bd_sent
27721,啊啊啊我看不来古装剧…… 但是又想看千玺 我真的哎,0.236483
3118,千玺加油，主要是编剧不咋地，有仿琅琊榜的痕迹，但仿不来啊,0.055295
21770,为何李必束发玉冠的簪子是前后走向，而不是左右走向的？,0.187549
30031,易烊千玺的演技与老戏骨肯定存在差距，但在他这个年纪，已经算很好了。,0.219752
10064,这易烊千玺说话怎么不看字幕听不清楚呢。,0.262272


In [195]:
data.groupby('episode')['bd_sent'].mean()

episode
1     0.572392
2     0.602381
3     0.629635
4     0.611072
5     0.626902
6     0.567174
7     0.575160
8     0.587230
9     0.544952
10    0.532704
11    0.568242
12    0.552880
13    0.509141
14    0.483916
15    0.489348
16    0.472495
Name: bd_sent, dtype: float64

In [198]:
data['date'] = data['bj_time'].dt.date

In [199]:
data.groupby('date')['bd_sent'].mean()

date
2019-06-27    0.586568
2019-06-28    0.614234
2019-06-29    0.594944
2019-06-30    0.573950
2019-07-01    0.546272
2019-07-02    0.565285
2019-07-03    0.570292
2019-07-04    0.512705
2019-07-05    0.505439
2019-07-06    0.518768
2019-07-07    0.530555
Name: bd_sent, dtype: float64